### 2．학습 데이터 준비

In [1]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# OS 패키지 임포트
import os
# Pillow 라이브러리 임포트
from PIL import Image
# NumPy 라이브러리 임포트
import numpy as np
# pandas 라이브러리 임포트
import pandas as pd
# scikit-learn 라이브러리 임포트
from sklearn import datasets, model_selection

In [8]:
# 대상 이미지 폴더 지정
dirs = ['ants', 'bees']

# 이미지의 픽셀값과 레이블을 저장할 리스트 초기화
data = [] # 설명변수
label = [] # 목적변수

# 각 폴더의 이미지 파일을 하나씩 읽어 들이여 가공하고 리스트에 추가
for i, d in enumerate(dirs):
    # 파일 목록 생성
    files = os.listdir('./data/hymenoptera_data/' + d)
    
    for f in files:
        # 이미지 읽어 들이기
        img = Image.open('./data/hymenoptera_data/' + d + '/' + f, 'r')
        # 128 * 128로 리사이징
        resize_img = img.resize((128, 128))
        # 채널별로 분리해서 [0,1] 구간으로 정규화
        resize_img.split()
        r_resize_img = np.asarray(np.float32(r)/255.0)
        g_resize_img = np.asarray(np.float32(g)/255.0)
        b_resize_img = np.asarray(np.float32(b)/255.0)
        rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
        # 가공된 이미지를 리스트에 추가
        data.append(rgb_resize_img)
        
        # 이미지 레이블을 리스트에 추가
        label.append(i)

In [ ]:
# 설명변수를 데이터프레임으로 변환해서 화면에 출력
pd.DataFrame(data[0][0]) # R채널

In [ ]:
# 목적변수를 화면에 출력
label

In [ ]:
# NumPy 배열로 변환
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

In [ ]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(
    data, label, test_size=0.1)

# 데이터 건수를 확인
print(len(train_X))
print(len(test_X))

### 3．텐서 생성

In [ ]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 텐서 크기를 화면에 출력
print(train_X.shape)
print(train_Y.shape)

In [ ]:
# 설명변수와 목적변수 텐서를 합치기
train = TensorDataset(train_X, train_Y)

# 첫 번째 텐서의 내용을 화면에 출력
print(train[0])

# 미니배치로 분할하기
train_loader = DataLoader(train, batch_size=32, shuffle=True)

### 4．신경망 구성

In [ ]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 합성곱층
        self.conv1 = nn.Conv2d(3, 10, 5) # 입력 채널 수, 출력 채널 수, 필터 크기
        self.conv2 = nn.Conv2d(10, 20, 5)

        # 전결합층
        self.fc1 = nn.Linear(20 * 29 * 29, 50) # 29=(((((128-5)+1)/2)-5)+1)/2
        self.fc2 = nn.Linear(50, 2)
    
    def forward(self, x):
        # 풀링층
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # 풀링 영역 크기
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 20 * 29 * 29)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

### 5．모형 학습

In [ ]:
# 오차함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    # 분할해 둔 데이터를 꺼내옴
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data[0]
    # 50회 반복마다 누적오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

In [ ]:
# NumPy 배열로 변환
test_X = np.array(test_X, dtype='float32')
test_Y = np.array(test_Y, dtype='int64')

# 테스트 데이터를 텐서로 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서 크기 확인
print(test_X.shape)
print(test_Y.shape)

In [ ]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)
# 출력이 0 혹은 1이 되게 함
result = torch.max(model(test_x).data, 1)[1]
# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy